# Step 6: “Advanced” Assignment

# Step 6.1 Pulling Data from the Web

In [18]:
import pandas as pd
import sqlite3

In [19]:
engine = sqlite3.connect('HW1_DB')

In [23]:
airports_df = pd.read_sql('select * from airports_df', engine)
airlines_df = pd.read_sql('select * from airlines_df', engine)
routes_df = pd.read_sql('select * from routes_df', engine)

In [22]:
flights_08 = pd.read_csv('http://big.dataanalytics.education/data/Flights-Aug-2016.csv')
flights_08.drop('Unnamed: 15', axis = 1, inplace = True)
flights_08.to_sql('flights', engine, if_exists = 'replace', index = False)

URLError: <urlopen error [Errno 99] Cannot assign requested address>

In [ ]:
# flights_08

In [24]:
flights_09 = pd.read_csv('http://big.dataanalytics.education/data/Flights-Sep-2016.csv')
flights_09.drop('Unnamed: 15', axis = 1, inplace = True)
flights_09.to_sql('flights', engine, if_exists = 'append', index = False)

URLError: <urlopen error [Errno 99] Cannot assign requested address>

In [7]:
flights_10 = pd.read_csv('http://big.dataanalytics.education/data/Flights-Oct-2016.csv')
flights_10.drop('Unnamed: 15', axis = 1, inplace = True)
flights_10.to_sql('flights', engine, if_exists = 'append', index = False)

In [8]:
import requests as req
import io
response_08 = req.get('http://big.dataanalytics.education/data/Flights-Aug-2016.csv')
aug_csv = io.StringIO(response_08.text)
aug_csv_df = pd.read_csv(aug_csv, usecols = ['Year', 'Month', 'DayofMonth', 'Carrier', 'FlightNum', 'Origin', 'Dest',\
                                           'DepTime', 'ArrTime', 'ArrDelayMinutes', 'Cancelled'], dtype = {'DepTime' : str, 'ArrTime' : str})

In [9]:
df = pd.read_sql('select count (*) from flights', engine)

In [10]:
int(df.ix[0, 0])

1425851

# Step 6.2 Comprehensive Flight Information Using SQL

# Step 6.2.1 Flights by Airline and Destination

In [11]:
iata_count_df = pd.read_sql(\
                                    'SELECT flights.Carrier, flights.Dest, count(flights.Origin)\
                                    FROM flights INNER JOIN routes_df ON flights.Carrier = routes_df.airline\
                                    AND flights.Dest = routes_df.destination_airport AND flights.Origin = routes_df.source_airport\
                                    GROUP BY flights.Carrier, flights.Dest ORDER BY flights.Carrier, flights.Dest LIMIT 60', engine)
iata_count_df.rename(columns = {'Carrier' : 'carrier_iata', 'Dest' : 'destination_airport_iata', 'count(flights.Origin)' : 'count'}, inplace = True)
iata_count_df.set_index(['carrier_iata', 'destination_airport_iata'], inplace = True)
# iata_count_df

# Step 6.2.2 Delayed Flights by Airline and Destination

In [12]:
delay_df = pd.read_sql(\
                                    'SELECT flights.Carrier, flights.Dest, count(flights.Origin)\
                                    FROM flights INNER JOIN routes_df ON flights.Carrier = routes_df.airline\
                                    AND flights.Dest = routes_df.destination_airport AND flights.Origin = routes_df.source_airport\
                                    WHERE flights.ArrDelayMinutes >= 30 OR flights.Cancelled == 1\
                                    GROUP BY flights.Carrier, flights.Dest ORDER BY flights.Carrier, flights.Dest LIMIT 60', engine)
delay_df.rename(columns = {'Carrier' : 'carrier_iata', 'Dest' : 'destination_airport_iata', 'count(flights.Origin)' : 'count'}, inplace = True)
delay_df.set_index(['carrier_iata', 'destination_airport_iata'], inplace = True)
# delay_df

# Step 6.2.3 Frequently-Delayed Flights by Airline and Destination

In [13]:
freq_delay_df = pd.read_sql(\
                                    'SELECT flights.Carrier, flights.Dest, count(flights.Origin)\
                                    FROM flights INNER JOIN routes_df ON flights.Carrier = routes_df.airline\
                                    AND flights.Dest = routes_df.destination_airport AND flights.Origin = routes_df.source_airport\
                                    WHERE flights.ArrDelayMinutes >= 30 OR flights.Cancelled == 1\
                                    GROUP BY flights.Carrier, flights.Dest \
                                    HAVING count(flights.Origin) >= 1000\
                                    ORDER BY flights.Carrier, flights.Dest LIMIT 60', engine)
freq_delay_df.rename(columns = {'Carrier' : 'carrier_iata', 'Dest' : 'destination_airport_iata', 'count(flights.Origin)' : 'count'}, inplace = True)
freq_delay_df.set_index(['carrier_iata', 'destination_airport_iata'], inplace = True)
# freq_delay_df

In [14]:
import matplotlib.pyplot as plt

entity_num = range(len(freq_delay_df))
bar_chart = plt.bar(entity_num, freq_delay_df['count'])
plt.xticks(entity_num, freq_delay_df.index.values, rotation = 90)
plt.xlabel('Label:(Airline IATA, Destination Airport IATA)')
plt.ylabel('Number of Bad Flights')
plt.title('Bar chart for bad flights from Aug. to Sept. in 2016'.title())
plt.show()

# Step 6.3 Shorter Travel Times?

In [15]:
shortest_single_hop_df = pd.read_sql(\
                                    'SELECT flights.Year, flights.Month, flights.DayofMonth, flights.Origin, flights.Dest, flights.DepTime, min(flights.ArrTime) AS ArrTime\
                                    FROM flights INNER JOIN routes_df ON flights.Carrier = routes_df.airline\
                                    AND flights.Dest = routes_df.destination_airport AND flights.Origin = routes_df.source_airport\
                                    WHERE flights.Year == 2016 AND flights.Month == 9 AND flights.ArrTime > flights.DepTime\
                                    GROUP BY flights.Year, flights.Month, flights.DayofMonth, flights.Origin, flights.Dest, flights.DepTime\
                                    ORDER BY flights.Year, flights.Month, flights.DayofMonth, flights.Origin, flights.Dest, flights.DepTime', engine)
shortest_single_hop_df.dropna(subset = ['Origin', 'Dest', 'DepTime', 'ArrTime'], inplace = True)
shortest_single_hop_df.to_sql('shortest', engine, if_exists = 'replace', index = False)
engine.execute('CREATE index my_index ON shortest (Year, Month, DayofMonth, Origin, Dest)')

In [16]:
# shortest_single_hop_df

In [17]:
shorter_df = pd.read_sql(\
                               'SELECT flights.Year AS Year, flights.Month AS Month, flights.DayofMonth AS DayofMonth, flights.DepTime AS Departure_Time, flights.Origin AS Origin, flights.Dest AS Dest, \
                                MIN(flights.ArrTime) AS Earliest_Arrival\
                                FROM flights JOIN routes_df ON flights.Dest = routes_df.destination_airport AND flights.Carrier = routes_df.airline AND flights.Origin = routes_df.source_airport_id\
                                WHERE flights.Month = 9 AND flights.Year = 2016 AND flights.DayofMonth IS NOT NULL AND flights.DepTime IS NOT NULL AND flights.ArrTime IS NOT NULL\
                                AND flights.Origin IS NOT NULL AND flights.Dest IS NOT NULL\
                                GROUP BY flights.DayofMonth, flights.DepTime, flights.Origin, flights.Dest', engine)
shorter_df.to_sql("shortest", engine, index = False, if_exists = 'replace')
shorter_df

,Year,Month,DayofMonth,Departure_Time,Origin,Dest,Earliest_Arrival
